Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [52]:
from itertools import accumulate
from random import random, seed
from itertools import product
import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from collections import deque
from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [53]:
UNIVERSE_SIZE = 100_000          ##Total number of element that must be covered
NUM_SETS = 10_000                ##Number of subset free to cover the universe
DENSITY = 0.1                    ##Probability of an element being in a set. With a density of 0.2, 20% of the universe will be in a set

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [54]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY ##If SETS[i, j] == True, then j is in set i, false otherwise
for s in range(UNIVERSE_SIZE):                               ##For evry element in the universe, check if it is in any set
    if not np.any(SETS[:, s]):                               ##If not, put it in a random set
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)                        ##Cost of each set


## Helper Functions

In [55]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## HILL CLIMBERS

In [56]:
MAX_TWEAKS = 3
NUM_RESTARTS = 1
MAX_ATTEMPTS = 30

all_solution_hashes = set()

In [57]:
##First inititialization of the fitness, used for every restart if the algorithm
def fitness_initialization(solution):
    return (valid(solution), -cost(solution))

##The invalid solution return a waighted cost, in order to prefer valid solutions
def fitness(solution, best_so_far):
    if valid(solution):
        return (True, -cost(solution))
    else:
        return (False, best_so_far*1.5) 

In [58]:
def single_tweak(solution: np.ndarray) -> np.ndarray:
    new_solution = solution.copy()
    i = rng.integers(0, NUM_SETS)
    new_solution[i] = not new_solution[i]
    return new_solution


def multiple_tweaks(solution: np.ndarray) -> np.ndarray:
    new_solution = solution.copy()
    for _ in range(rng.integers(1, MAX_TWEAKS)):
        i = rng.integers(0, NUM_SETS)
        new_solution[i] = not new_solution[i]
    return new_solution

##Try to avoid using the same solution of the past tweaks
def tabu_or_multiple_tweak(solution: np.ndarray) -> np.ndarray:
    attempts = 0
    new_solution = solution.copy()

    while attempts < MAX_ATTEMPTS:
        mask = rng.random(NUM_SETS) < 0.02
        new_solution = np.logical_xor(solution, mask)
            
        # Calculate the hash of the new solution 
        sol_hash = hash(new_solution.tobytes())
        # Check if the hash is already in the set of solutions
        if sol_hash not in all_solution_hashes:
            all_solution_hashes.add(sol_hash)  
            return new_solution
        
        attempts += 1

    #If exit from loop after max_attempts, return the random solution. (No change)
    mask = rng.random(NUM_SETS) < 0.5
    new_solution = np.logical_xor(solution, mask)
        
    return new_solution



In [ ]:
##Hill climbing with restart
tweak = tabu_or_multiple_tweak
history = list()

for r in tqdm(range(NUM_RESTARTS), position = 0):
    #Evry restart of the algorithm, a new solution is generated
    solution = rng.random(NUM_SETS) <  np.random.uniform(0.2, 1.0)
    
    solution_fitness = fitness_initialization(solution)
    history.append(solution_fitness[1])
    all_solution_hashes.clear()

    ic(solution_fitness)

    for steps in tqdm(range(10_000), position= 1):
        new_solution = tweak(solution)
        fit = fitness(new_solution, solution_fitness[1])
        history.append(fit[1])
                                      #In this way i avoid the zero solution that it's not valid  
        if fit > solution_fitness and solution_fitness[1] < 0:
            solution = new_solution
            solution_fitness = fit
            

    ic(r, solution_fitness)


plt.figure(figsize=(14, 8))
plt.plot(
    range(len(history)),
    list(accumulate(history, max)),
    color="red",
)
_ = plt.scatter(range(len(history)), history, marker=".")



    
